# Ciencia de Datos en Python
## Proyecto #1

### Creacion de entorno virtual en Anaconda 

In [1]:
# conda create --name regression_1 python=3.8 numpy pandas matplotlib seaborn scikit-learn
# conda activate regression_1

In [11]:
# Carga de librerias 
import numpy as np
import pandas as pd

In [13]:
# Cargar datos
data = np.load('proyecto_training_data.npy')

# Calcular el índice para dividir los datos
split_index = int(len(data) * 0.8)

# Dividir datos en entrenamiento y validación
train_data = data[:split_index]
test_data = data[split_index:]

# Verificando las dimensiones de los conjuntos de datos
print(f'Dataset completo: {data.shape}')
print(f'Set de entrenamiento: {train_data.shape}')
print(f'Set de validación: {test_data.shape}')

Dataset completo: (1460, 6)
Set de entrenamiento: (1168, 6)
Set de validación: (292, 6)
